In [0]:
# %sql
# CREATE TABLE IF NOT EXISTS claims_transcriptions (
#     file_id STRING GENERATED ALWAYS AS (sha2(concat(file_path, modificationTime), 256)),
#     file_path STRING,
#     modificationTime TIMESTAMP,
#     name STRING,
#     size BIGINT,
#     transcription STRING
# ) USING DELTA;

# INSERT INTO claims_transcriptions
# SELECT sha2(concat(file_path, modificationTime), 256), file_path, modificationTime, name, size, transcription
# FROM source_table;


In [0]:
%sql
-- Create the table for claims analysis if it doesn't exist
CREATE TABLE IF NOT EXISTS samantha_wise.ai_claims_processing.claims_analysis (
    sentiment STRING,
    topic STRING,
    full_name STRING,
    date_of_birth STRING,
    post_code STRING,
    processing_time TIMESTAMP
) USING DELTA;

-- Insert results from transcription analysis
INSERT INTO samantha_wise.ai_claims_processing.claims_analysis
SELECT
    ai_analyze_sentiment(transcription) AS sentiment,
    ai_extract_topics(transcription) AS topic,
    ai_extract(transcription, 'PERSON') AS full_name,
    ai_extract(transcription, 'DATE') AS date_of_birth,
    ai_extract(transcription, 'GPE') AS post_code,
    current_timestamp() AS processing_time
FROM samantha_wise.ai_claims_processing.transcriptions;

In [0]:
%sql
SELECT 
    get_json_object(topic, '$.topic') AS topic_name, 
    COUNT(*) AS topic_count
FROM samantha_wise.ai_claims_processing.transcriptions
GROUP BY topic_name
ORDER BY topic_count DESC;